<a href="https://colab.research.google.com/github/Brahmi5459/AI-Resume-Builder/blob/main/RAG_IMplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os

from groq import Groq

from langchain_groq import ChatGroq

# Set the GROQ_API_KEY environment variable
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

llm = ChatGroq(
    model="llama3-8b-8192",
    # You no longer need to pass the api_key here, as it's now set in the environment variable
    # api_key=os.environ.get("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")
)

In [8]:
!pip install langchain_groq

In [10]:
!pip install langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.9 MB/s eta 0:00:00


In [11]:
!pip install langchain_huggingface

In [12]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [13]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.6 MB/s eta 0:00:00


In [15]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader

# Define the website URL you want to load
website_url = "https://pricehistory.app/"  # Replace with your actual website URL

# Use WebBaseLoader to load the content of the website
web_loader = WebBaseLoader(website_url)
documents = web_loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# Create a vector store using Hugging Face embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieve and generate using the relevant snippets of the document
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm  # Make sure you have defined `llm` elsewhere in your code
    | StrOutputParser()
)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [19]:
# Invoke the RAG chain with a question
response = rag_chain.invoke("What is the Price of samsung mobile?")
print(response)

I don't know the price of Samsung mobile as the provided context does not mention Samsung mobile.


In [20]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What is list?")

retrieved_docs

[Document(metadata={'description': '\uf8ffüìàPrice History is a free price tracker tool to check price history information for millions of products. Our Amazon Price Tracker tracks Prices, Lightning deals, and Coupon Discounts. Flipkart Price Tracker tracks Product Price History & Tap and Save discounts. Check Price Graphs, Offer Prices on Important Sale Events, Additional Offer Discounts and Get Price drop alerts with Price History', 'language': 'en', 'source': 'https://pricehistory.app/', 'title': 'Price History: Price Tracker for Amazon, Flipkart & more'}, page_content='‚Çπ99          Price History       AMAZON INDIA          Highest     ‚Çπ3515       Lowest ‚Çπ1203    Borosil Fluted Cascade Floral Opalware Dinner Set, (25-Pieces, White)    Price: ‚Çπ2742          Price History       AMAZON INDIA          Highest     ‚Çπ3545       Lowest ‚Çπ2499    Crompton Energion Hyperjet 1200mm BLDC Ceiling Fan | Point Anywhere Remote Control | BEE 5 Star Rated Energy Efficient | Superior Air De

In [21]:
print(retrieved_docs[0].page_content)

‚Çπ99          Price History       AMAZON INDIA          Highest     ‚Çπ3515       Lowest ‚Çπ1203    Borosil Fluted Cascade Floral Opalware Dinner Set, (25-Pieces, White)    Price: ‚Çπ2742          Price History       AMAZON INDIA          Highest     ‚Çπ3545       Lowest ‚Çπ2499    Crompton Energion Hyperjet 1200mm BLDC Ceiling Fan | Point Anywhere Remote Control | BEE 5 Star Rated Energy Efficient | Superior Air Delivery | Anti-Rust | 2 Year Manufacturer Warranty | White Black    Price: ‚Çπ2499          Price History       AMAZON INDIA          Highest     ‚Çπ399       Lowest ‚Çπ192    Lifelong Yoga Blocks - Dual Colour Yoga Bricks Pack of 1 for Men & Women, Balancing Yoga Bricks with EVA Foam & Soft Surface, Triple Layer for Balance, Support & Cushion-Exercise Equipment for Home    Price: ‚Çπ192          Price History       FLIPKART INDIA          Highest     ‚Çπ2799       Lowest ‚Çπ1199    Anker 10000 mAh Power Bank (Black, Lithium Polymer, Fast Charging for Mobile)    Price:


In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [23]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is list give an example?"):
    print(chunk, end="", flush=True)

I don't know. There is no explicit example given in the provided context.

In [25]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does you know python progamming?"})
print(response["answer"])

I don't know.


In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is python?"})
print(response["answer"])

I don't know the answer to that question. Python is a programming language, but the context I was provided doesn't mention Python.


In [28]:
for document in response["context"]:
    print(document)
    print()

page_content='LCD Writing Tablet Pad with Screen 21.5cm (8.5Inch) for Drawing, Playing, Handwriting Best Birthday Gifts for Adults & Kids Girls Boys, Multicolor    Price: ‚Çπ98          Price History       AMAZON INDIA          Highest     ‚Çπ1716       Lowest ‚Çπ459    Halonix 20-watt LED Batten/Tubelight | Streak square 4-ft LED Batten for Living Room & Bedroom | Cool Day Light, Pack of 4    Price: ‚Çπ459          Price History       AMAZON INDIA          Highest     ‚Çπ1716       Lowest ‚Çπ459    Halonix 20-watt LED Batten/Tubelight | Streak square 4-ft LED Batten for Living Room & Bedroom | Cool Day Light, Pack of 4    Price: ‚Çπ459          Price History       AMAZON INDIA          Highest     ‚Çπ5999       Lowest ‚Çπ722    Amazon Basics Outdoor Textilene Adjustable Zero Gravity Folding Reclining Lounge Chair with Pillow (Blue) | Alloy Steel    Price: ‚Çπ5099          Price History       AMAZON INDIA          Highest     ‚Çπ630       Lowest ‚Çπ132    Nootie Dog Plush Leash Medium 

In [29]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is AI Resume Builder?")

"I don't know the answer to this question because it's not related to the provided context about Price History App and its features."